# Calibration of for LDM

In [ ]:
%matplotlib widget

In [ ]:
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

## Determine the time step size

In [ ]:
def get_step_cr(**kw):
    f_t, E_w = kw['f_t'], kw['E_w']
    bartau, E_s = kw['bartau'], kw['E_s']
    w_cr = f_t / E_w
    s_cr = bartau / E_s
    return w_cr, s_cr

## Material parameters

Define the parameters that are close to the original MARS setup
and test if the stress-slip/opening response is comparable.
Also see if the dissipated energies are in a realistic range.

In [ ]:
E_w = 60000
alpha = 0.3
E_s = alpha * E_w
f_t = 2
material_params = dict(
    E_s=E_s, gamma_s=0, K_s=0, S_s=0.000005, c_s=.1, bartau=6, 
    E_w=E_w, S_w=0.0000000065, c_w = 0.001, m = 0.01, f_t=2, f_c=50, f_c0=20, eta=0
)

@MV - parameters that lead to interrupted iteration

In [ ]:
E_w = 80000
alpha = 0.3
E_s = alpha * E_w

f_t = 4.4
material_params = dict(
    E_s=E_s, gamma_s=10, K_s=10, S_s=0.000048, c_s1=0.1, bartau=3*f_t,
    E_w=E_w, S_w=0.00000065, c_w = 0.001, m = 0.1, f_t=f_t, f_c=50, f_c0=40, eta=1
)

w_cr, s_cr = get_step_cr(**material_params)
w_max = w_cr * 35
s_max = s_cr * 3
w_cr, w_max, s_cr, 20*s_max

Define the algorithmic parameters, like the step size and maximum value so that the 
transition between elastic and inelastic range is crossed with a small step

@todo: define a time function that adapts the step size based on this transition.

In [ ]:
n_steps = 1000
# Too much:
calib_mtx = [
#     [0, 0, w_max],
#    [0, 0, 0.008],
     [20*s_max, 0, 0],
#     [s_max, 0, w_max],
#     [s_max, s_max, w_max]
]

In [ ]:
se = SlideExplorer(n_steps=n_steps, k_max=100)
se.slide_model.trait_set(**material_params);

Elementary loading combined scenarios 

In [ ]:
%%capture
for s_x_1, s_y_1, w_1 in calib_mtx:
    print('running', s_cr, s_x_1, s_y_1, w_1)
    se.reset()
    se.trait_set(s_x_1=s_x_1, s_y_1=s_y_1, w_1=w_1, n_steps=n_steps);
    se.run()

In [ ]:
bu.InteractiveWindow([se, se.energy_dissipation, se.inel_state_evolution, 
                      se.slide_model], figsize=(9,4), tight_layout=False).interact()

@todo: define scenarios with unloading to distinguish the plastic and damage dissipation
in response to normal and tangential loading